## Агрегирование по MCC коду

### Основная гипотеза

Могут быть более мужские (шиномонтаж) и более женские (косметика) транзакции. 

Соотношение транзакций (70/30, 50/50, 0/100) может нам дать понимание гендера клиента.

### Проверка гипотезы

На прошлом шаге мы получили сет с транзакциями

In [47]:
import pandas as pd

transactions = pd.read_pickle('datasets/formatted_set.pkl')
transactions.sample(3, random_state=11)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender,tr_description,device,cash_relation,region,type,mcc_description,mcc_full_description,mcc_group,amount_abs,is_income,tr_date,tr_time,tr_week
4852146,24634142,2014-09-14 16:39:16,5411,1110,-9515.95,25011898,NaN,Покупка. POS ТУ Россия,POS,unrelated,other_bank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,9515.95,0.0,2014-09-14,16:39:16,37
3368757,4891744,2014-08-08 19:00:02,5411,1010,-875.91,323492,NaN,Покупка. POS ТУ СБ РФ,POS,unrelated,sberbank_trx,POS покупка,Бакалейные магазины (5411),"Бакалейные магазины, супермаркеты",54,875.91,0.0,2014-08-08,19:00:02,32
6191439,29894123,2014-09-16 11:01:26,6010,7070,49499.98,888987,NaN,Перевод на карту (с карты) через Мобильный бан...,SMS,unrelated,unrelated,B2C перевод,Финансовые институты (6010),Финансовые институты — снятие наличности вручную,60,49499.98,1.0,2014-09-16,11:01:26,38


#### Сборка сета МСС

Соберем данные по количеству тразнакций по каждой МСС в разрезе гендеров.

In [48]:
# Создаем таблицу, состоящую из транзакций из POS терминалов (со словом "Покупка" в описании)
pivot = transactions.copy()
pivot = pivot[pivot['type']=='POS покупка']                                 # оставляем только операции с POS терминалов
pivot = pivot[pivot['gender'].notna()]                                      # оставляем только транзации с указанием пола
pivot = pivot[['gender', 'mcc_description']]                                # оставляем только необходимые столбцы для анализа 

# энкодим и переименовываем значения гендеров
pivot = (
    pd.get_dummies(pivot, columns=['gender'], dtype='int')
    .rename(columns={'gender_0.0':'f_trx', 'gender_1.0':'m_trx'})
)

# группируем транзакции по МСС
pivot = (pivot.groupby(by=['mcc_description']).sum())

# добавляем общее количество транзакций для МСС и показатель соотношения по гендерам 
pivot['total_trx'] = pivot.f_trx + pivot.m_trx                              # считаем общее количество транзакций
pivot['m_day_ratio'] = pivot.m_trx / (pivot.m_trx + pivot.f_trx) * 2 - 1    # считаем долю мужских транзакций
pivot = pivot.sort_values(by='total_trx', ascending=False)                  # сортируем

pivot

,f_trx,m_trx,total_trx,m_day_ratio
mcc_description,,,,
Бакалейные магазины (5411),287465,250977,538442,-0.067766
Различные продовольственные магазины (5499),99234,89193,188427,-0.053289
Станции техобслуживания (5541),21322,58958,80280,0.468809
Аптеки (5912),48349,27806,76155,-0.269752
Универсальные магазины (5331),40668,32399,73067,-0.113170
...,...,...,...,...
Услуги по переделке (5697),14,10,24,-0.166667
Разнообразные издательства (2741),10,12,22,0.090909
Финансовые институты (6010),8,7,15,-0.066667


Показатель `m_day_ratio` принимает значения от -1 (если все транзакции в этой МСС совершены мужчинами) до 1 (если все транзакции совершены женщинами).

Если `m_day_ratio` равно нулю, значит в МСС одинаковое количество операций совершенных обоими гендерами.

Дополним таблицу статистикой того какая доля клиентов имеет операции в этой МСС. Для этого составим таблицу с данными по проникновению клиентов в МСС.

In [49]:
# добавляем подсчет количества клиентов и доли от их общего количества
mcc_clients = transactions.copy()
mcc_clients = mcc_clients[mcc_clients['type']=='POS покупка'] # фильтруем транзакции с POS терминалов

# считаем количество клиентов с операциями в этой МСС
mcc_clients = (
    mcc_clients[['customer_id', 'mcc_description']]
    .groupby(by=['mcc_description', 'customer_id']).count()
)
mcc_clients = mcc_clients.reset_index()
mcc_clients = mcc_clients.groupby(by=['mcc_description']).count()
mcc_clients.columns = ['cl_count']

# Считаем долю клиентов с операциями в этой МСС от общего количества клиентов
clients_count = len(transactions.customer_id.value_counts().index)
mcc_clients['cl_share'] = mcc_clients.cl_count / clients_count
mcc_clients = mcc_clients.reset_index()


# сливаем данные по транзакциям и по клиентам
pivot = pd.merge(pivot, mcc_clients, on='mcc_description', how='left')
pivot

,mcc_description,f_trx,m_trx,total_trx,m_day_ratio,cl_count,cl_share
0,Бакалейные магазины (5411),287465,250977,538442,-0.067766,12960,0.864000
1,Различные продовольственные магазины (5499),99234,89193,188427,-0.053289,10030,0.668667
2,Станции техобслуживания (5541),21322,58958,80280,0.468809,7166,0.477733
3,Аптеки (5912),48349,27806,76155,-0.269752,10668,0.711200
4,Универсальные магазины (5331),40668,32399,73067,-0.113170,8315,0.554333
...,...,...,...,...,...,...,...
178,Услуги по переделке (5697),14,10,24,-0.166667,37,0.002467
179,Разнообразные издательства (2741),10,12,22,0.090909,33,0.002200
180,Финансовые институты (6010),8,7,15,-0.066667,24,0.001600
181,Агенты и менеджеры по аренде недвижимости (6513),10,5,15,-0.333333,22,0.001467


Мы получили данные по каждой MCC с указанием:
- количества транзакций по ней (мужских, женских и всего)
- соотношения мужских и женских операций
- количества клиентов, совершивших операции в этой МСС 
- долю клиентов с операциями в МСС от их общего количества